In [37]:
import sys
sys.path.insert(0, "../app/coman")
import os
from dotenv import load_dotenv
from langchain_community.vectorstores.azuresearch import AzureSearch, AzureSearchVectorStoreRetriever
from langchain_openai import AzureOpenAIEmbeddings
from ActuaDocumentLoader import ActuaDocumentLoader
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
)

In [38]:
load_dotenv()

AZURE_SEARCH_KEY = str(os.getenv("AZURE_SEARCH_KEY"))
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="orisai-text-embedding-3-large-development",
)

fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embeddings.embed_query("Text")),
        vector_search_profile_name="myHnswProfile",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchableField(
        name="source",
        type=SearchFieldDataType.String,
        searchable=False,
        filterable=True,
    ),
    SearchableField(
        name="test",
        type=SearchFieldDataType.String,
        searchable=False,
        filterable=True,
    )
]

index_name: str = "test-index-tom"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint="https://orisai-search-development.search.windows.net",
    azure_search_key=AZURE_SEARCH_KEY,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    fields=fields
)

In [39]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever.invoke("wat is de brusselse woonfiscaliteit")

[Document(page_content='Brusselse woonfiscaliteit wordt op 1 januari 2023 hervormd <p>Brussels minister van Financiën Sven Gatz stelt een fiscale hervorming in het vooruitzicht voor het jaareinde. Daarbij wordt gesleuteld aan het abattement in de registratierechten. Het doel is om de aankoop van de eerste gezinswoning goedkoper te maken. De teksten liggen momenteel voor bij diverse adviesraden en moeten vervolgens nog naar het Brussels parlement.&nbsp;</p> <p>Volgens de informatie die minister Gatz wereldkundig heeft gemaakt bestaat de hervorming uit drie wijzigingen aan het abattement op de registratierechten. We herinneren kort hoe die regeling vandaag in elkaar zit.</p><h3><strong>Wat is het abattement?</strong></h3><p>Het abattement is gelijk te stellen aan een belastingvoetvrijstelling van €175.000. Concreet mag een koper die €175.000 aftrekken van de aankoopprijs om vervolgens op het restant het tarief van 12,5% toe te passen. Het fiscaal voordeel bedraagt zo maximaal €21.875. We

In [62]:
retriever = AzureSearchVectorStoreRetriever(vectorstore=vector_store, k=2)


retriever.invoke("wat is de brusselse woonfiscaliteit", filters="source eq '1234'")

[]

In [ ]:

loader = ActuaDocumentLoader("https://dev-coman-api.immo-connect.be/v1/odata/contentitem?$count=true&$orderby=DateCreated%20desc&$skip=0&$top=25&$expand=Values,Scheme&$filter=(Scheme/Id%20eq%201482edab-dac9-4400-bcd7-ab2dd28b96d2)%20and%20(Status%20eq%20%27Active%27)")

documents = loader.lazy_load()
print(documents)
for document in documents:
    print(document.metadata)

In [ ]:
vector_store.add_documents(documents)

In [32]:
docs = vector_store.vector_search("Wanneer wordt Brusselse woonfiscaliteit hervormd?", k=5, filters="source eq '547c540c-af51-4bbb-9476-1291977ebe1a'",
    score_threshold=0.5)
print(docs)

[Document(page_content='Uit het parlement: kan indexering huurprijzen gekoppeld worden aan energieprestatie? <p>De hoge inflatie is in sterke mate toe te schrijven aan de almaar stijgende energieprijzen. De energieprijzen doen ook de gezondheidsindex stijgen, die wordt gehanteerd als basis voor de indexering van de huurprijzen. Huurders krijgen dus tweemaal de rekening gepresenteerd. Een voorstel van Comeos, die de Belgische handel en diensten vertegenwoordigt, beoogt dit recht te trekken.</p> <p>In de Commissie voor Wonen en Onroerend Erfgoed van het Vlaamse Parlement vormde het voorstel van Comeos om de indexering te koppelen aan de energiescore van het pand voer voor debat. In het voorstel van Comeos hangt de mate waarin indexering kan toegepast worden af van de EPC-score van het verhuurde pand. Voor eigendommen die goed scoren, minstens een A-score, kan de volledige indexatie doorgerekend worden. Voor eigendommen die minder energie-efficiënt scoren zal de energiecomponent uit de ge